## Minhash LSH Implementation

In [210]:
#pip install datasketch recordlinkage

In [211]:
import numpy as np
import pandas as pd
import re
import time
import string
import random
from datasketch import MinHash, MinHashLSH
from recordlinkage.datasets import load_febrl1, load_febrl2, load_febrl3

In [212]:
# df = load_febrl3()
# csv_file_path = 'febrl3_processed_with_text.csv'
# df.to_csv(csv_file_path, index=True, encoding='utf-8')

In [213]:
df = load_febrl2()
exclude_columns = ['soc_sec_id', 'street_number']
df_processed = df.fillna('')
cols_to_drop = [col for col in exclude_columns if col in df_processed.columns]
merged_column = df_processed.drop(columns=cols_to_drop).apply(lambda x: ' '.join(x.astype(str)), axis=1)
df["text"] = merged_column
csv_file_path = 'febrl2_processed_with_text.csv'
df.to_csv(csv_file_path, index=True, encoding='utf-8')

In [214]:
df.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,text
rec_id,,,,,,,,,,,
rec-2778-org,sarah,bruhn,44,forbes street,wintersloe,kellerberrin,4510,vic,19300213,7535316,sarah bruhn forbes street wintersloe kellerber...
rec-712-dup-0,jacob,lanyon,5,milne cove,wellwod,beaconsfield upper,2602,vic,19080712,9497788,jacob lanyon milne cove wellwod beaconsfield u...
rec-1321-org,brinley,efthimiou,35,sturdee crescent,tremearne,scarborough,5211,qld,19940319,6814956,brinley efthimiou sturdee crescent tremearne s...
rec-3004-org,aleisha,hobson,54,oliver street,inglewood,toowoomba,3175,qld,19290427,5967384,aleisha hobson oliver street inglewood toowoom...
rec-1384-org,ethan,gazzola,49,sheaffe street,bimby vale,port pirie,3088,sa,19631225,3832742,ethan gazzola sheaffe street bimby vale port p...


In [215]:
N_SHINGLE = 5
NUM_PERM = 128
def tokenize(text, n):
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = []
    for i in range(len(text) - n + 1):
        tokens.append(text[i:i+n])
    return tokens

def preprocess(text, n_shingle=N_SHINGLE):
    remove_chars = string.punctuation + '@.'
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', remove_chars))
    tokens = tokenize(text, n_shingle)
    return tokens

def get_minhash(text, num_perm=NUM_PERM, n_shingle=N_SHINGLE):
    tokens = preprocess(text, n_shingle)
    m = MinHash(num_perm=num_perm)
    for token in tokens:
        m.update(token.encode('utf8'))
    return m

In [216]:
INDEX_A = 3
INDEX_B = 290
text_A = df['text'].iloc[INDEX_A]
text_B = df['text'].iloc[INDEX_B]
shingles_A = set(preprocess(text_A))
shingles_B = set(preprocess(text_B))

print(f"Number of Shingles A({INDEX_A}): {len(shingles_A)}")
print(f"Number of Shingles B({INDEX_B}): {len(shingles_B)}")

Number of Shingles A(3): 62
Number of Shingles B(290): 58


In [217]:
intersection = len(shingles_A.intersection(shingles_B))
union = len(shingles_A.union(shingles_B))

J_actual = intersection / union
print(f"Actual Jaccard similarity: {J_actual:.4f}")

Actual Jaccard similarity: 0.0256


In [218]:
m_A = get_minhash(text_A)
m_B = get_minhash(text_B)
J_minhash_estimated = m_A.jaccard(m_B)
print(f"Estimated Jaccard similarity (MinHash): {J_minhash_estimated:.4f}")
print(f"\nDifference: {abs(J_minhash_estimated - J_actual):.4f}")

Estimated Jaccard similarity (MinHash): 0.0391

Difference: 0.0134


In [219]:
THRESHOLD = 0.70
start_time = time.time()
df['minhash_signature'] = df['text'].apply(lambda x: get_minhash(x, NUM_PERM, N_SHINGLE))
time_taken = time.time() - start_time
print(f"Completed creating MinHash signatures for {len(df)} records in {time_taken:.2f} seconds.")
print(f"Signature size: {len(df['minhash_signature'].iloc[0].hashvalues)}")

Completed creating MinHash signatures for 5000 records in 4.48 seconds.
Signature size: 128


In [220]:
df.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,text,minhash_signature
rec_id,,,,,,,,,,,,
rec-2778-org,sarah,bruhn,44,forbes street,wintersloe,kellerberrin,4510,vic,19300213,7535316,sarah bruhn forbes street wintersloe kellerber...,<datasketch.minhash.MinHash object at 0x78191e...
rec-712-dup-0,jacob,lanyon,5,milne cove,wellwod,beaconsfield upper,2602,vic,19080712,9497788,jacob lanyon milne cove wellwod beaconsfield u...,<datasketch.minhash.MinHash object at 0x78191e...
rec-1321-org,brinley,efthimiou,35,sturdee crescent,tremearne,scarborough,5211,qld,19940319,6814956,brinley efthimiou sturdee crescent tremearne s...,<datasketch.minhash.MinHash object at 0x78191e...
rec-3004-org,aleisha,hobson,54,oliver street,inglewood,toowoomba,3175,qld,19290427,5967384,aleisha hobson oliver street inglewood toowoom...,<datasketch.minhash.MinHash object at 0x78191e...
rec-1384-org,ethan,gazzola,49,sheaffe street,bimby vale,port pirie,3088,sa,19631225,3832742,ethan gazzola sheaffe street bimby vale port p...,<datasketch.minhash.MinHash object at 0x78191e...


In [221]:
lsh = MinHashLSH(threshold=THRESHOLD, num_perm=NUM_PERM)
start_time = time.time()
print(f"Starting MinHashLSH index construction with threshold {THRESHOLD}...")

for rec_id, m_sig in df['minhash_signature'].items():
    lsh.insert(rec_id, m_sig)

time_taken = time.time() - start_time
print(f'LSH Indexing completed in {time_taken:.4f} seconds.')
try:
    QUERY_ID = 'rec-10-org' 
    m_query = df.loc[QUERY_ID, 'minhash_signature']
    
    candidate_keys = lsh.query(m_query)
    
    print(f"LSH found {len(candidate_keys)} potential candidates.")

    if candidate_keys:
        for key in candidate_keys:
            record = df.loc[key] 
            jaccard_est = m_query.jaccard(record['minhash_signature'])
            print(f"Key (rec_id) {key} | Jaccard Est: {jaccard_est:.4f}")
            print(f"  > Name: {record['given_name']} {record['surname']}")
            
except KeyError:
    print("Could not find rec_id 'rec-0-org'. Please ensure index is correct.")

Starting MinHashLSH index construction with threshold 0.7...
LSH Indexing completed in 0.2875 seconds.
LSH found 1 potential candidates.
Key (rec_id) rec-10-org | Jaccard Est: 1.0000
  > Name: lucy crouch


In [222]:
def brute_force(df, threshold=THRESHOLD):
    start_time = time.time()
    signatures = df['minhash_signature'].tolist()
    N = len(signatures)
    candidate_pairs = set()
    
    print(f"\nStarting O(N^2) brute-force comparison for {N} records...")
    
    for i in range(N):
        for j in range(i + 1, N):
            jaccard_est = signatures[i].jaccard(signatures[j])
            if jaccard_est >= threshold:
                candidate_pairs.add(tuple(sorted((i, j))))
                
    time_taken = time.time() - start_time
    print(f"Brute-force completed: {len(candidate_pairs)} pairs found in {time_taken:.4f} seconds.")
    return candidate_pairs, time_taken

def true_brute_force(df, threshold=THRESHOLD):
    start_time = time.time()
    shingle_sets = [set(preprocess(text, N_SHINGLE)) for text in df['text']]
    N = len(shingle_sets)
    candidate_pairs = set()
    
    print(f"\nStarting exact Jaccard brute-force comparison for {N} records...")
    
    for i in range(N):
        for j in range(i + 1, N):
            set_a = shingle_sets[i]
            set_b = shingle_sets[j]
            
            intersection = len(set_a.intersection(set_b))
            union = len(set_a.union(set_b))
            
            if union > 0:
                jaccard_actual = intersection / union
            else:
                jaccard_actual = 0.0
            
            if jaccard_actual >= threshold:
                candidate_pairs.add(tuple(sorted((i, j))))
                
    time_taken = time.time() - start_time
    print(f"Exact brute-force completed: {len(candidate_pairs)} pairs found in {time_taken:.4f} seconds.")
    return candidate_pairs, time_taken

def lsh_candidate_search(df, lsh_index):
    start_time = time.time()
    candidate_pairs = set()
    
    for key_idx, m_sig in df['minhash_signature'].items():
        candidates = lsh_index.query(m_sig)
        current_int_idx = df.index.get_loc(key_idx)
        
        for candidate_key in candidates:
            candidate_int_idx = df.index.get_loc(candidate_key)
            
            if key_idx != candidate_key: 
                candidate_pairs.add(tuple(sorted((current_int_idx, candidate_int_idx))))
                
    time_taken = time.time() - start_time
    return candidate_pairs, time_taken

true_pairs, true_time = true_brute_force(df, threshold=THRESHOLD)
lsh_pairs, lsh_time = lsh_candidate_search(df, lsh)

true_positives = len(lsh_pairs.intersection(true_pairs))

precision = true_positives / len(lsh_pairs) if len(lsh_pairs) > 0 else 0.0
recall = true_positives / len(true_pairs) if len(true_pairs) > 0 else 0.0

print("\nREALITY CHECK EVALUATION")
print(f"Total True Pairs (Exact Jaccard >= {THRESHOLD}): {len(true_pairs)}")
print(f"Total LSH Candidates Found: {len(lsh_pairs)}")
print(f"True Positives (Intersection): {true_positives}")

print(f"\nPRECISION: {precision:.4f}")
print(f"RECALL: {recall:.4f}")

print("PERFORMANCE BENCHMARK")
print(f"Exact Brute-Force Time: {true_time:.4f} seconds")
print(f"LSH Search Time: {lsh_time:.4f} seconds")

if true_time > 0 and lsh_time > 0:
    print(f"LSH is {true_time / lsh_time:.1f} times faster than Exact Brute-Force.")

   




Starting exact Jaccard brute-force comparison for 5000 records...
Exact brute-force completed: 695 pairs found in 48.4433 seconds.

REALITY CHECK EVALUATION
Total True Pairs (Exact Jaccard >= 0.7): 695
Total LSH Candidates Found: 700
True Positives (Intersection): 554

PRECISION: 0.7914
RECALL: 0.7971
PERFORMANCE BENCHMARK
Exact Brute-Force Time: 48.4433 seconds
LSH Search Time: 0.0882 seconds
LSH is 549.1 times faster than Exact Brute-Force.
